In [1]:
import pandas as pd
import numpy as np
import time

start_time = time.time()
url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz'
url_data = pd.read_table(url, sep='\t', nrows=200000, error_bad_lines=False)
print("--- %s seconds ---" % (time.time() - start_time))
#url_data = pd.read_table(url, nrows=100000)

b'Skipping line 9076: expected 15 fields, saw 22\nSkipping line 19256: expected 15 fields, saw 22\nSkipping line 24313: expected 15 fields, saw 22\nSkipping line 47211: expected 15 fields, saw 22\nSkipping line 54295: expected 15 fields, saw 22\nSkipping line 56641: expected 15 fields, saw 22\nSkipping line 63067: expected 15 fields, saw 22\n'
b'Skipping line 93796: expected 15 fields, saw 22\n'
b'Skipping line 132806: expected 15 fields, saw 22\nSkipping line 164631: expected 15 fields, saw 22\nSkipping line 167019: expected 15 fields, saw 22\nSkipping line 167212: expected 15 fields, saw 22\n'
b'Skipping line 198103: expected 15 fields, saw 22\nSkipping line 199191: expected 15 fields, saw 22\n'


--- 72.21122598648071 seconds ---


In [2]:
url_data

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,41409413,R2MTG1GCZLR2DK,B00428R89M,112201306,yoomall 5M Antenna WIFI RP-SMA Female to Male ...,Electronics,5,0,0,N,Y,Five Stars,As described.,2015-08-31
1,US,49668221,R2HBOEM8LE9928,B000068O48,734576678,"Hosa GPM-103 3.5mm TRS to 1/4"" TRS Adaptor",Electronics,5,0,0,N,Y,It works as advertising.,It works as advertising.,2015-08-31
2,US,12338275,R1P4RW1R9FDPEE,B000GGKOG8,614448099,Channel Master Titan 2 Antenna Preamplifier,Electronics,5,1,1,N,Y,Five Stars,Works pissa,2015-08-31
3,US,38487968,R1EBPM82ENI67M,B000NU4OTA,72265257,LIMTECH Wall charger + USB Hotsync & Charging ...,Electronics,1,0,0,N,Y,One Star,Did not work at all.,2015-08-31
4,US,23732619,R372S58V6D11AT,B00JOQIO6S,308169188,Skullcandy Air Raid Portable Bluetooth Speaker,Electronics,5,1,1,N,Y,Overall pleased with the item,Works well. Bass is somewhat lacking but is pr...,2015-08-31
5,US,21257820,R1A4514XOYI1PD,B008NCD2LG,976385982,Pioneer SP-BS22-LR Andrew Jones Designed Books...,Electronics,5,1,1,N,Y,Five Stars,The quality on these speakers is insanely good...,2015-08-31
6,US,3084991,R20D9EHB7N20V6,B00007FGUF,670878953,C2G/Cables to Go 03170 3.5mm F/F Stereo Coupler,Electronics,5,0,0,N,Y,Lifesaver,Wish I could give this product more than five ...,2015-08-31
7,US,8153674,R1WUTD8MVSROJU,B00M9V2RMM,508452933,COOLEAD-HDMI Switcher BOX,Electronics,5,0,0,N,Y,Five Stars,works great,2015-08-31
8,US,52246189,R1QCYLT25812DM,B00J3O9DYI,766372886,Philips Wireless Portable Speaker,Electronics,4,0,0,N,Y,Four Stars,Great sound and compact. Battery life seems go...,2015-08-31
9,US,41463864,R904DQPBCEM7A,B00NS1A0E4,458130381,PlayStation 3 3D Glasses (Super Value 4 Pack),Electronics,4,0,0,N,Y,Four Stars,It works well~~~,2015-08-31


In [3]:
url_data.groupby('product_id').count()

,marketplace,customer_id,review_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
product_id,,,,,,,,,,,,,,
0511189877,8,8,8,8,8,8,8,8,8,8,8,8,8,8
0594296420,4,4,4,4,4,4,4,4,4,4,4,4,4,4
0594451647,1,1,1,1,1,1,1,1,1,1,1,1,1,1
059445204X,1,1,1,1,1,1,1,1,1,1,1,1,1,1
097185548X,1,1,1,1,1,1,1,1,1,1,1,1,1,1
0972683275,40,40,40,40,40,40,40,40,40,40,40,40,40,40
1400501776,2,2,2,2,2,2,2,2,2,2,2,2,2,2
1400532620,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1400532655,5,5,5,5,5,5,5,5,5,5,5,5,5,5
